# Environement Setup

In [5]:
# @title 1. 🛠️ Installazione Ambiente (PyTorch 2.4.0)
import os
import sys

print("🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)")

# 1. Disinstallazione versioni incompatibili
os.system("pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-geometric")

# 2. Installazione PyTorch 2.4.0 (Versione richiesta dal README)
# Usiamo CUDA 12.1 che è stabile su Colab
cmd_torch = "pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121"
os.system(cmd_torch)

# 3. Installazione librerie Grafiche (PyG) specifiche per Torch 2.4.0
pyg_url = "https://data.pyg.org/whl/torch-2.4.0+cu121.html"
libs = ["torch-scatter", "torch-sparse", "torch-cluster", "torch-spline-conv", "torch-kmeans"]
cmd_pyg = f"pip install {' '.join(libs)} -f {pyg_url}"
os.system(cmd_pyg)

# 4. Altre dipendenze
os.system("pip install torch-geometric hydra-core omegaconf submitit pandas ftfy")

os.system("pip install -r requirements.txt")

print("✅ Installazione completata.")
print("⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!")

🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)
✅ Installazione completata.
⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!


In [1]:
# @title 2. 📂 Setup HiERO e Download Pesi Pre-addestrati
import os

# 1. Clone del repository
if not os.path.exists("HiERO"):
    print("⬇️ Clonazione repository HiERO...")
    os.system("git clone --recursive https://github.com/MarcoPernoVDP/HiERO-main.git")

os.chdir("/content/HiERO-main")

# 2. Creazione struttura cartelle
dirs = [
    "checkpoints",
    "pretrained",
    "data/ego4d/raw/annotations",
    "data/ego4d/raw/features/egovlp"
]
for d in dirs:
    os.makedirs(d, exist_ok=True)

# 3. Download del modello HiERO già addestrato (Backbone EgoVLP)
if not os.path.exists("checkpoints/hiero_egovlp.pth"):
    print("⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...")
    os.system("wget -O checkpoints/hiero_egovlp.pth https://www.sapeirone.it/data/hiero/ckpt/hiero_egovlp.pth")

# 4. Download componenti testuali (richiesti dal codice)
print("⬇️ Scaricamento Encoder Testuale...")
os.system("wget -q -O pretrained/egovlp_text.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_text.pth")
os.system("wget -q -O pretrained/egovlp_txt_proj.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_txt_proj.pth")

# 5. Download Annotazioni necessarie (EgoMCQ/EgoClip)
print("⬇️ Scaricamento Annotazioni...")
os.system("gdown --id 1-aaDu_Gi-Y2sQI_2rsI2D1zvQBJnHpXl -O data/ego4d/raw/annotations/egoclip.csv")
os.system("gdown --id 1-5iRYf4BCHmj4MYQYFRMY4bhsWJUN3rW -O data/ego4d/raw/annotations/egomcq.json")

print("✅ Setup completato. Cartelle pronte.")

⬇️ Clonazione repository HiERO...
⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...
⬇️ Scaricamento Encoder Testuale...
⬇️ Scaricamento Annotazioni...
✅ Setup completato. Cartelle pronte.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
import zipfile

zip_path = "/content/drive/MyDrive/MistakeDetection/egovlp_test.zip"
extract_path = "/content/data/egovlp"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ZIP estratto correttamente")

Mounted at /content/drive
✅ ZIP estratto correttamente


In [1]:
# Prima vai nella directory del repo
%cd /content/HiERO-main

# Poi esegui lo script
!python infer.py \
  --checkpoint /content/HiERO-main/checkpoints/hiero_egovlp.pth \
  --input /content/data \
  --output /content/outputs/embeddings \
  --num_clusters 8 \
  --device cuda

/content/HiERO-main
tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 223kB/s]
config.json: 100% 483/483 [00:00<00:00, 2.91MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 1.59MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 5.22MB/s]
config.json: 100% 483/483 [00:00<00:00, 2.29MB/s]
2025-12-19 22:48:07.754069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766184488.108579    4290 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766184488.210684    4290 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766184488.920702    4290 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same

In [24]:
import os
import glob
import torch
import numpy as np
from tqdm import tqdm
from omegaconf import OmegaConf
from sklearn.cluster import KMeans
from torch_geometric.data import Data

# ==========================================
# 1. SETUP PERCORSI
# ==========================================
USER_ACTUAL_PATH = "/content/data/egovlp/egovlp_test"
HIERO_ROOT = "/content/HiERO"
OUTPUT_DIR = "/content/output_embeddings"
os.makedirs(OUTPUT_DIR, exist_ok=True)

import sys
if HIERO_ROOT not in sys.path:
    sys.path.append(HIERO_ROOT)
os.chdir(HIERO_ROOT)

print(f"📂 Cartella Input Dati: {USER_ACTUAL_PATH}")
print(f"📂 Cartella Output: {OUTPUT_DIR}")

# ==========================================
# 2. CONFIGURAZIONE MODELLO
# ==========================================
print("\n⚙️ Configurazione Modello...")
from models.hiero import HiERO

conv_config = OmegaConf.create({
    "_target_": "models.conv.tdgc.TDGC",
    "in_channels": 768, "out_channels": 768, "bias": True,
    "pos_hidden_channels": 256, "aggr": "mean", "pos_act": "tanh",
    "use_signed_temporal_distance": True, "use_pos_attr": True
})

model = HiERO(
    input_size=256, hidden_size=768, conv=conv_config,
    depth=3, n_clusters=8, input_projection=True, dropout=0.1,
    pool="batch_subsampling"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ckpt_path = os.path.join(HIERO_ROOT, "checkpoints/hiero_egovlp.pth")

checkpoint = torch.load(ckpt_path, map_location="cpu")
state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
model.load_state_dict(state_dict, strict=False)
model = model.to(device).eval()
print(f"✅ Modello pronto su {device}")

# ==========================================
# 3. CERCA FILE NPZ
# ==========================================
files_npz = glob.glob(f"{USER_ACTUAL_PATH}/*.npz")
print(f"📂 Trovati {len(files_npz)} file .npz.")

# ==========================================
# 4. PROCESSA OGNI VIDEO
# ==========================================
count = 0
with torch.no_grad():
    for fpath in tqdm(files_npz):
        filename = os.path.basename(fpath)
        out_name = filename.replace('.npz', '.pth')
        save_path = os.path.join(OUTPUT_DIR, out_name)

        try:
            # --- CARICAMENTO FEATURE ---
            with np.load(fpath, allow_pickle=True) as data:
                keys = list(data.keys())
                if 'features' in keys: feat_np = data['features']
                elif 'arr_0' in keys: feat_np = data['arr_0']
                else: feat_np = data[keys[0]]

            feat = torch.from_numpy(feat_np).float()
            if feat.dim() == 2:
                feat = feat.unsqueeze(0)  # (1, T, F_dim)
            B, T, F_dim = feat.shape
            if T == 0: continue
            print(f"{filename} -> B: {B}, T: {T}, F_dim: {F_dim}")

            # --- CREAZIONE DATA PER HiERO ---
            x_flat = feat.reshape(-1, F_dim)
            pos = torch.arange(T, dtype=torch.float).repeat(B)
            batch_idx = torch.arange(B).repeat_interleave(T)
            indices = torch.arange(T).repeat(B)

            data_obj = Data(
                x=x_flat,
                pos=pos,
                batch=batch_idx,
                indices=indices
            )
            # maschera necessaria per TDGC
            data_obj.mask = torch.ones(x_flat.shape[0], dtype=torch.bool)

            # --- INFERENZA HiERO ---
            data_obj = data_obj.to(device)
            out_batch = model(data_obj)
            final_emb = out_batch.x  # (B*T, hidden_dim)

            # --- CLUSTERING IN 8 STEP ---
            emb_np = final_emb.cpu().numpy()
            kmeans = KMeans(n_clusters=8, random_state=0).fit(emb_np)
            clusters = kmeans.labels_

            # --- SALVATAGGIO ---
            torch.save({
                "embeddings": final_emb.cpu(),
                "clusters": torch.tensor(clusters)
            }, save_path)
            count += 1

        except Exception as e:
            print(f"⚠️ Errore su {filename}: {e}")

print(f"\n✅ OPERAZIONE COMPLETATA! Salvati {count} file in: {OUTPUT_DIR}")


📂 Cartella Input Dati: /content/data/egovlp/egovlp_test
📂 Cartella Output: /content/output_embeddings

⚙️ Configurazione Modello...


/tmp/ipython-input-2098950513.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location="cpu")


✅ Modello pronto su cuda
📂 Trovati 10 file .npz.


 40%|████      | 4/10 [00:00<00:00, 37.49it/s]

3_18_360p_224.mp4_1s_1s.npz -> B: 1, T: 1789, F_dim: 256
⚠️ Errore su 3_18_360p_224.mp4_1s_1s.npz: 2
1_14_360p_224.mp4_1s_1s.npz -> B: 1, T: 1329, F_dim: 256
⚠️ Errore su 1_14_360p_224.mp4_1s_1s.npz: 2
3_5_360p_224.mp4_1s_1s.npz -> B: 1, T: 1091, F_dim: 256
⚠️ Errore su 3_5_360p_224.mp4_1s_1s.npz: 2
3_13_360p_224.mp4_1s_1s.npz -> B: 1, T: 1654, F_dim: 256
⚠️ Errore su 3_13_360p_224.mp4_1s_1s.npz: 2
3_12_360p_224.mp4_1s_1s.npz -> B: 1, T: 1237, F_dim: 256
⚠️ Errore su 3_12_360p_224.mp4_1s_1s.npz: 2
1_32_360p_224.mp4_1s_1s.npz -> B: 1, T: 679, F_dim: 256
⚠️ Errore su 1_32_360p_224.mp4_1s_1s.npz: 2
1_10_360p_224.mp4_1s_1s.npz -> B: 1, T: 887, F_dim: 256
⚠️ Errore su 1_10_360p_224.mp4_1s_1s.npz: 2


100%|██████████| 10/10 [00:00<00:00, 33.42it/s]

3_14_360p_224.mp4_1s_1s.npz -> B: 1, T: 1132, F_dim: 256
⚠️ Errore su 3_14_360p_224.mp4_1s_1s.npz: 2
1_19_360p_224.mp4_1s_1s.npz -> B: 1, T: 1301, F_dim: 256
⚠️ Errore su 1_19_360p_224.mp4_1s_1s.npz: 2
2_28_360p_224.mp4_1s_1s.npz -> B: 1, T: 2055, F_dim: 256
⚠️ Errore su 2_28_360p_224.mp4_1s_1s.npz: 2

✅ OPERAZIONE COMPLETATA! Salvati 0 file in: /content/output_embeddings


In [3]:
!ls -R /content/data/egovlp/egovlp_test

ls: cannot access '/content/data/egovlp/egovlp_test': No such file or directory
